### Creating a binary classifier to detect smiles

In its most basic form, image classification consists of discerning between two classes, or signaling the presence or absence of some trait. In this recipe, we'll implement a binary classifier that tells us wether a person in a photo is smiling.

As we are working inside Colab we need to install wget to pass the SMILEs dataset from GDrive to Colab

In [2]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=53bcfaf5101f2c1331b7bf7b79195f83a92bd013a80d2e01e47d5988e741cb95
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
# Import all necessary packages

import os
import pathlib
import wget
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *

In [4]:
# Define a function to load the images and labels from a list of file paths

def load_images_and_labels(image_paths):
  images = []
  labels = []

  for image_path in image_paths:
    image = load_img(image_path, target_size=(32,32),
                     color_mode='grayscale')
    image = img_to_array(image)

    label = image_path.split(os.path.sep)[-2]
    label = 'positive' in label
    label = float(label)

    images.append(image)
    labels.append(label)

  return np.array(images), np.array(labels)


In [6]:
# Define a function to build the neural network.
# This model's structure is based on LeNet

def build_network():
  input_layer = Input(shape=(32,32,1))
  x = Conv2D(filters=20,
             kernel_size=(5,5),
             padding='same',
             strides=(1,1))(input_layer)
  x = ELU()(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(pool_size=(2,2),
                   strides=(2,2))(x)
  x = Dropout(0.4)(x)

  x = Conv2D(filters=50,
             kernel_size=(5,5),
             padding='same',
             strides=(1,1))(x)
  x = ELU()(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(pool_size=(2,2),
                   strides=(2,2))(x)
  x = Dropout(0.4)(x)

  x = Flatten()(x)
  x = Dense(units=500)(x)
  x = ELU()(x)
  x = Dropout(0.4)(x)

  output = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input_layer, outputs=output)
  return model